In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve

import sweetviz as sv
import dtale
import pandas_profiling

from xverse.transformer import WOE

In [2]:
ar = pd.read_csv('G:/Rahul Data/GCD Related/INSAID/Dataset/application_record.csv')
cr = pd.read_csv('G:/Rahul Data/GCD Related/INSAID/Dataset/credit_record.csv')

print(ar.head())
print('\n')
print(cr.head())

        ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0  5008804           M            Y               Y             0   
1  5008805           M            Y               Y             0   
2  5008806           M            Y               Y             0   
3  5008808           F            N               Y             0   
4  5008809           F            N               Y             0   

   AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0          427500.0               Working               Higher education   
1          427500.0               Working               Higher education   
2          112500.0               Working  Secondary / secondary special   
3          270000.0  Commercial associate  Secondary / secondary special   
4          270000.0  Commercial associate  Secondary / secondary special   

     NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  DAYS_BIRTH  DAYS_EMPLOYED  \
0        Civil marriage   Rented apartment      -12005 

In [3]:
df1 = pd.DataFrame(cr.groupby(['ID','STATUS']).agg('count'))

In [4]:
df1.reset_index(inplace=True)
df1.head()

,ID,STATUS,MONTHS_BALANCE
0,5001711,0,3
1,5001711,X,1
2,5001712,0,10
3,5001712,C,9
4,5001713,X,22


In [5]:
def condition(x):
    if ((x=='1')|(x=='2')|(x=='3')|(x=='4')|(x=='5')):
        return 1
    else:
        return 0

df1['Delinquent'] = df1['STATUS'].apply(condition)

In [6]:
df1[df1['ID']==5001718]

,ID,STATUS,MONTHS_BALANCE,Delinquent
9,5001718,0,24,0
10,5001718,1,2,1
11,5001718,C,3,0
12,5001718,X,10,0


In [7]:
df2 = pd.DataFrame(df1.groupby(['ID']).sum('Delinquent'))

In [8]:
df2.reset_index(inplace=True)
df2.head(25)

,ID,MONTHS_BALANCE,Delinquent
0,5001711,4,0
1,5001712,19,0
2,5001713,22,0
3,5001714,15,0
4,5001715,60,0
5,5001717,22,0
6,5001718,39,1
7,5001719,43,0
8,5001720,36,1
9,5001723,31,0


In [9]:
df2.drop('MONTHS_BALANCE',axis=1,inplace=True)
df2.head(25)

,ID,Delinquent
0,5001711,0
1,5001712,0
2,5001713,0
3,5001714,0
4,5001715,0
5,5001717,0
6,5001718,1
7,5001719,0
8,5001720,1
9,5001723,0


In [10]:
final = pd.merge(ar,df2,'inner',on='ID')
final.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,Delinquent
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0


In [11]:
final[final['Delinquent'] >= 1]

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,Delinquent
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1
18,5008825,F,Y,N,0,130500.0,Working,Incomplete higher,Married,House / apartment,-10669,-1103,1,0,0,0,Accountants,2.0,1
19,5008826,F,Y,N,0,130500.0,Working,Incomplete higher,Married,House / apartment,-10669,-1103,1,0,0,0,Accountants,2.0,1
20,5008830,F,N,Y,0,157500.0,Working,Secondary / secondary special,Married,House / apartment,-10031,-1469,1,0,1,0,Laborers,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5149828,M,Y,Y,0,315000.0,Working,Secondary / secondary special,Married,House / apartment,-17348,-2420,1,0,0,0,Managers,2.0,2
36453,5149834,F,N,Y,0,157500.0,Commercial associate,Higher education,Married,House / apartment,-12387,-1325,1,0,1,1,Medicine staff,2.0,5
36454,5149838,F,N,Y,0,157500.0,Pensioner,Higher education,Married,House / apartment,-12387,-1325,1,0,1,1,Medicine staff,2.0,5
36455,5150049,F,N,Y,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,-17958,-655,1,0,0,0,Sales staff,2.0,2


In [12]:
def DelRes(x):
    if x>= 1:
        return 1
    else:
        return 0

final['Delinquent'] = final['Delinquent'].apply(DelRes)

In [13]:
y = final['Delinquent']
X = final.drop('Delinquent',axis=1)

In [14]:
X.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [15]:
y.head()

0    1
1    1
2    0
3    0
4    0
Name: Delinquent, dtype: int64

In [16]:
print('Shape of X is ',format(X.shape))

Shape of X is  (36457, 18)


In [17]:
print('Shape of y is ',format(y.shape))

Shape of y is  (36457,)


In [18]:
clf = WOE()
clf.fit(X,y)

WOE(mono_custom_binning={'AMT_INCOME_TOTAL': array([  27000.,  135000.,  202500., 1575000.]),
                         'CNT_CHILDREN': array([ 0.,  1., 19.]),
                         'CNT_FAM_MEMBERS': array([ 1.,  2., 20.]),
                         'DAYS_BIRTH': array([-25152., -18085., -13545.,  -7489.]),
                         'DAYS_EMPLOYED': array([-15713.,  -2512.,   -762., 365243.]),
                         'FLAG_EMAIL': array([0., 1.]),
                         'FLAG_MOBIL': array([1.]),
                         'FLAG_PHONE': array([0., 1.]),
                         'FLAG_...
                                  'Low-skill Laborers': 0.5550706919292876,
                                  'Managers': 0.10884426286411482,
                                  'Medicine staff': 0.15732562042824635,
                                  'NA': -0.11623544902968029,
                                  'Private service staff': -0.6691189041220243,
                                  'Realty age

In [19]:
X = clf.transform(X)

In [20]:
X.drop('ID',axis=1,inplace=True)
X.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,0.090813,-0.042655,-0.062117,-0.016592,0.092374,-0.017516,-0.012749,0.064983,0.191859,0.130236,0.011547,0.0,-0.000857,0.012689,-0.019294,-0.116235,-0.009197
1,0.090813,-0.042655,-0.062117,-0.016592,0.092374,-0.017516,-0.012749,0.064983,0.191859,0.130236,0.011547,0.0,-0.000857,0.012689,-0.019294,-0.116235,-0.009197
2,0.090813,-0.042655,-0.062117,-0.016592,-0.041559,-0.017516,-0.010246,-0.013198,-0.014551,-0.079759,0.031467,0.0,0.000250,0.012689,-0.019294,0.308644,-0.009197
3,-0.047125,0.025439,-0.062117,-0.016592,0.092374,0.088521,-0.010246,0.104669,-0.014551,-0.079759,0.011547,0.0,0.000250,-0.030863,0.181365,-0.059897,-0.009197
4,-0.047125,0.025439,-0.062117,-0.016592,0.092374,0.088521,-0.010246,0.104669,-0.014551,-0.079759,0.011547,0.0,0.000250,-0.030863,0.181365,-0.059897,-0.009197


In [21]:
X['AMT_INCOME_TOTAL'].unique()

array([ 0.09237436, -0.04155892, -0.04637093])

In [26]:
adb = AdaBoostClassifier(base_estimator=RandomForestClassifier(),n_estimators=50,learning_rate=1,random_state=42)

In [27]:
from sklearn.metrics import plot_confusion_matrix

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=10)

adb.fit(X_train,y_train)

adb.predict(X_test)

plot_confusion_matrix(AdaBoostClassifier(base_estimator=RandomForestClassifier(),n_estimators=50,learning_rate=1,random_state=42),X_train,y_train,)


print('R2 Score of Training Data After Applying Adaptive Boostin is : {}',format(adb.score(X_train,y_train)))
print('\n')
print('R2 Score of Testing Data After Applying Adaptive Boostin is : {}',format(adb.score(X_test,y_test)))

In [60]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=10)

parm_rang = np.arange(1,30,5)

train_score,test_score = validation_curve(RandomForestClassifier(), X_train,y_train,param_name='n_estimators',param_range=parm_rang,cv=3,scoring='accuracy',n_jobs=1)

train_mean = np.mean(train_score,axis=1)
train_std = np.std(train_score,axis=1)

test_mean = np.mean(test_score,axis=1)
test_std = np.std(test_score,axis=1)

plt.figure(figsize=(15,7))
plt.plot(parm_rang,train_mean,label="Training Score")
plt.plot(parm_rang,test_mean,label="Testing Score")
plt.xlabel('N Estimators')
plt.ylabel('Accuracy Score')
plt.title('N Estimators Vs Accuracy')
plt.legend(loc='best')

In [61]:
parm_rang = np.arange(1,30,5)

train_score,test_score = validation_curve(RandomForestClassifier(), X_train,y_train,param_name='n_estimators',param_range=parm_rang,cv=3,scoring='accuracy',n_jobs=1)

train_mean = np.mean(train_score,axis=1)
train_std = np.std(train_score,axis=1)

test_mean = np.mean(test_score,axis=1)
test_std = np.std(test_score,axis=1)

plt.figure(figsize=(15,7))
plt.plot(parm_rang,train_mean,label="Training Score")
plt.plot(parm_rang,test_mean,label="Testing Score")

In [63]:
par = {'n_estimators' : [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]}

search = GridSearchCV(RandomForestClassifier(),param_grid=par,scoring='accuracy',n_jobs=1,cv=3)
search.fit(X_train,y_train)
search.predict(X_test)

search.best_params_

{'n_estimators': 28}

In [64]:
search.cv_results_

{'mean_fit_time': array([0.04431693, 0.06494077, 0.09328763, 0.11709388, 0.15021563,
        0.17019176, 0.19935695, 0.22990863, 0.24609478, 0.25813007,
        0.30238255, 0.33058016, 0.35464525, 0.38543153, 0.41312011,
        0.44781454, 0.46013212, 0.57983621, 0.52857312, 0.54055206,
        0.5598712 , 0.60176889, 0.63676103, 0.64216328, 0.67875393,
        0.70971831, 0.76480214, 0.77241794, 0.78354096, 0.85107589]),
 'std_fit_time': array([0.00225896, 0.00194685, 0.00475827, 0.00593178, 0.00413569,
        0.00369406, 0.00750698, 0.00795743, 0.00265113, 0.01112504,
        0.00661512, 0.00345766, 0.00168252, 0.00473519, 0.01626535,
        0.00674611, 0.00666461, 0.07650464, 0.02024324, 0.01047896,
        0.01608896, 0.00763734, 0.01119547, 0.00240429, 0.00210646,
        0.01482481, 0.01157451, 0.03761356, 0.01299576, 0.01894247]),
 'mean_score_time': array([0.01333912, 0.01406272, 0.01828647, 0.02250544, 0.02760824,
        0.02979541, 0.03358603, 0.03746827, 0.04102961, 0.04

In [65]:
par_rag = np.arange(1,30,1)

train_score,test_score = validation_curve(RandomForestClassifier(),X_train,y_train,param_name='n_estimators',param_range=par_rag,cv=3,scoring='accuracy',n_jobs=1)

train_mean = np.mean(train_score,axis=1)
train_std = np.std(train_score,axis=1)

test_mean = np.mean(test_score,axis=1)
test_std = np.std(test_score,axis=1)

plt.figure(figsize=(15,7))
plt.plot(par_rag,train_mean,label='Training Accuracy Score',color='b')
plt.plot(par_rag,test_mean,label='Validation R2 Score',color='g')


plt.legend(loc='best')
plt.xlabel('max_depth')
plt.ylabel('R2 Score')
plt.title('Max Depth Vs R2 Score Plot')

plt.show()

In [67]:
rfc = RandomForestClassifier(n_estimators=28)
rfc.fit(X_train,y_train)
y_pred_test = rfc.predict(X_test)
y_pred_train = rfc.predict(X_train)

print('Training Classification Report : ')
print(classification_report(y_train,y_pred_train))
print('\n')
print('Testing Classification Report : ')
print(classification_report(y_test,y_pred_test))

Training Classification Report : 
              precision    recall  f1-score   support

           0       0.93      0.98      0.96     25692
           1       0.78      0.47      0.59      3473

    accuracy                           0.92     29165
   macro avg       0.86      0.73      0.77     29165
weighted avg       0.91      0.92      0.91     29165



Testing Classification Report : 
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      6474
           1       0.48      0.27      0.34       818

    accuracy                           0.89      7292
   macro avg       0.70      0.61      0.64      7292
weighted avg       0.86      0.89      0.87      7292



In [72]:
rfc = RandomForestClassifier(n_estimators=28)
rfc.fit(X_train,y_train)
plot_confusion_matrix(rfc,X_train,y_train)
plt.show()
print('\n')
plot_confusion_matrix(rfc,X_test,y_test)
plt.show()